In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [13]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [14]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [15]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [16]:
fake['class']=0
true['class']=1

In [17]:
data = pd.concat([fake,true], axis = 0)

In [18]:
data.sample(10)

,title,text,subject,date,class
3100,Kerry Washington Just Delivered A PERFECT One...,On the red carpet at Sunday s Golden Globe Awa...,News,"January 8, 2017",0
7988,White House calls Senate override of veto of S...,ABOARD AIR FORCE ONE (Reuters) - The White Hou...,politicsNews,"September 28, 2016",1
11238,"Trump, Cruz tensions may provide liveliest Rep...","CHARLESTON, S.C./MANCHESTER, N.H. (Reuters) - ...",politicsNews,"January 14, 2016",1
4139,Trump to order U.S. Treasury to delve into tax...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 20, 2017",1
22146,Episode #4 – DRIVE BY WIRE: ‘DC Rabbit Holes’ ...,MEMBERS can join co-hosts Patrick Henningsen ...,US_News,"March 30, 2017",0
13417,"In pre-recorded video, detained Venezuela oil ...",CARACAS (Reuters) - Venezuela s former Oil Min...,worldnews,"November 30, 2017",1
8431,WATCH: Bill Maher Explains How Republicans Ge...,"At some point, Republicans learned that they c...",News,"January 30, 2016",0
21797,(AUDIO) RACIST BLACK PANTHER LEADER: “We’re wi...,What is it that they don t get? Does anyone ev...,left-news,"Apr 30, 2015",0
20602,Employees Say FACEBOOK IS SUPPRESSING CONSERVA...,We know firsthand what it feels like to be a ...,left-news,"May 9, 2016",0
1344,Trump’s Bizarre Behavior At G7 Leaves The Wor...,While the other six world leaders at the G7 (G...,News,"May 27, 2017",0


In [19]:
data = data.drop(["title","subject","date"], axis = 1)

In [21]:
data.reset_index(inplace=True)

In [22]:
data.drop(['index'],axis=1,inplace=True)

In [23]:
data.sample(5)

,text,class
35038,SEOUL (Reuters) - U.S. President Donald Trump ...,1
41650,LONDON (Reuters) - British Prime Minister Ther...,1
6337,Millions of Americans across the country have ...,0
38581,UNITED NATIONS (Reuters) - South Sudan Preside...,1
4221,Liberty University President Jerry Falwell Jr....,0


In [26]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'\[.*?\]', '', text)  # Remove text in brackets
  text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Fix URL regex
  text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
  text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
  text = re.sub(r"\n", " ", text)  # Replace newlines with spaces
  text = re.sub(r"\d+", "", text)  # Remove numbers
  text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
  return text

In [27]:
data["text"] = data["text"].apply(clean_text)

In [29]:
x= data["text"]
y= data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25, random_state=42)

In [39]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [40]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [41]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9857461024498887

In [42]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [43]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']